In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import pearsonr
from pathlib import Path 

Se hace hace una grafica de correlación con las categorias de Minpoint H, usando cada una de las carpetas que hay ECOINVENT 

In [ ]:
Categorias = [
    "A_agri.xlsx",
    "B_mining.xlsx",
    "C10_food.xlsx",
    "C13_textil.xlsx",
    "C16_wood.xlsx",
    "C17_paper.xlsx",
    "C18_printing.xlsx",
    "C19_petroleum.xlsx",
    "C22_plastic.xlsx",
    "C24_metals.xlsx",
    "D351_electricity.xlsx",
    "H_transport.xlsx",
    "Recycled.xlsx"
]

for i in Categorias: 
    # 1. Importar archivo Excel
    archivo_excel = i 
    data = pd.read_excel(archivo_excel, header=None)

    # 2. Extraer nombres y datos
    names = data.iloc[:, 0].tolist()  # primera columna
    lci = data.iloc[:, 1:].dropna(how='all')

    # 3. Asegurar que todos los datos sean numéricos
    lci = lci.apply(pd.to_numeric, errors='coerce')

    # 4. Número de filas en lci
    nd = len(lci)

    # 5. Número de columnas (por fila original)
    num_columns = data.shape[1] 

    # 6. Calcular matriz de correlación
    corr_matrix = lci.corr(method='pearson')  # alternativa simple a `Outer[Correlation]`

    # 7. Visualizar matriz con seaborn
    plt.figure(figsize=(10, 8))
    sns.heatmap(corr_matrix, cmap='coolwarm', annot=True, vmin=corr_matrix.min().min(), vmax=corr_matrix.max().max(),
                xticklabels=range(1, 19), yticklabels=range(1, 19))
    plt.title(f"Correlation Matrix: {i}")
    plt.show()

    # Convertir la matriz a tabla larga (pares de variables)
    corr_pairs = corr_matrix.unstack().reset_index()
    corr_pairs.columns = ["Variable_1", "Variable_2", "Correlation"]

    # Eliminar duplicados (matriz es simétrica) y auto-correlaciones (corr=1)
    corr_pairs = corr_pairs[corr_pairs["Variable_1"] < corr_pairs["Variable_2"]]

    # Filtrar condiciones: mayor a 0.9 o menor a 0.25
    filtered_corr = corr_pairs[
        (corr_pairs["Correlation"] > 0.9) | (corr_pairs["Correlation"] < 0.25)
    ]
    # Agregar columna con el nombre del archivo (sin .xlsx si quieres)
    nombre = Path(i).stem  # sin extensión

    # Mostrar tabla con título
    display.display(filtered_corr.style.set_caption(f"Correlaciones destacadas - {nombre}"))
